Code by Marvin Kinz, m.kinz@stud.uni-heidelberg.de
# Surface Capturing

## Initial capturing process
Captures visible cutting surfaces from different perspectives by loading the initial and deformed slices, sampling the cutting surfaces and then capturing the visible sampled points as point clouds.

Creates border data as well, which can be used for recapturing, if other views or other numbers of samples are wanted. The "capture" function has to be run before "CleanUpUE4Data.ipynb", as it depends on the vertex data supplied by UE4, including duplicate vertices.

In [ ]:
import numpy as np
import vedo
import open3d as o3d
import glob
from pathlib import Path  
import time
import os
import csv
def capture(folder_in,folder_out,CameraPositions,sample_nr=5000,min_per=5,show=0,tri_threshold=3):
    """
    Captures from different perspectives visible cutting surfaces by loading the initial and deformed slices, sampling the cutting surfaces and then capturing the visible sampled points as point clouds. 

    ATTENTION:  Does not work, if the data has been made unique with Downsampling.ipynb. So run this capture function first, afterwards use recapture, if new captures are needed. 
                For more than 10 files the string handling when saving has to be improved, to not access positions, but extract the numbers, as accessing the positions stops working with double digit numbers.    

    params
    ------
    folder_in : string
        Folder where the folders Slices and Slices deformed are placed in.
    folder_out : string
        Folder to store the resulting data.
    sample_nr : int [1,inf)
        Number of sampled points per cutting surface.
    min_per : float [0,100)
        Minimum needed visible surface in percent of whole surface for capturing point cloud from view.
    CameraPositions : Array/List (N,3)
        Array of [X, Y, Z] camera positions to capture from, focus is always on object. Only depends on the relation between the components, i.e. [100,0,0] is the same as [1,0,0] and [100,10,0] is the same as [10,1,0]. 
    show : bool
        Set true if you want a visualization of the process. Press "w" in the viewer to enable the wireframe.
    tri_threshold : int
        Vertices that are in more than this number of triangles get counted as belonging to a cut surface. You can try to set this higher if there is disconnected spots sampled and lower if there is surface missing when sampled, but hopefully you will not have to.
   

    returns
    -------
    areas : array double
        Areas of partial captures
    bna : array double
        Broad-Narrow parameter for partial captures
    Stores sampled point clouds as .xyz files with information with further information on incomplete captures in a .csv file in specified output folder.
    Also stores border vertices of the cuts as .xyz together with .triangle and .normals data.
    """

    #create output directory, if it does not exist yet
    if not os.path.exists(folder_out):
        os.makedirs(folder_out)
    if not os.path.exists(f"{folder_out}/Views"):
        os.makedirs(f"{folder_out}/Views")
    areas=[] #to make histogram of captured areas
    bna=[] #to make histogram of broad narrow parameter
    #Lists to store additional data for views in csv file
    csv_header=["Filename","Target (Filename of other side of cut)", "Number of slice", "Camera Position", "Area", "Broad Narrow parameter"]
    csv_data=[]

    #go through all objects in input folder
    for obj in glob.glob(f"{folder_in}/Slices/*.xyz"):
        tic = time.perf_counter() #take time
        #load vertices initial, triangles, normals deformed and vertices deformed
        filename=Path(obj).stem
        vert=np.loadtxt(obj)
        tri=np.loadtxt(f"{folder_in}/Slices/{filename}.triangle")
        norm=np.loadtxt(f"{folder_in}/Slices deformed/{filename}.normals")
        vert_def=np.loadtxt(f"{folder_in}/Slices deformed/{filename}.xyz")

        #create mesh from deformed vertices and normals, triangles will be set later
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(vert_def)
        mesh.vertex_normals=o3d.utility.Vector3dVector(norm)

        #only sample on cut surfaces by determining vertices with more than a certain number of triangles
        values, counts = np.unique(tri, return_counts=True) #count how often each vertice is in triangle data
        counts_argsort=np.argsort(counts) #sort counts and return indices
        numtri=np.sum(counts>tri_threshold) #number of vertices in more than a certain number of triangles. You can try to set this higher if there is disconnected spots sampled and lower if there is surface missing when sampled.

        #fill list with indices of all triangles that countain those vertices
        list0=[] 
        for d in range(numtri):
            max_indice=int(values[counts_argsort[-d-1]])
            list0=np.concatenate((list0,np.argwhere(tri==max_indice)[:,0]))
        list0=np.unique(list0).astype(int)

        #Determine to what cutting surface the triangles belong by checking the y position of the undeformed vertices, because the cutting normal is in y direction
        tric=tri[list0][:,0]
        vertc=vert[tric.astype(int)][:,1]
        meanc=np.mean(vert,axis=0)
        maskc=vertc>meanc[1]
        
        #Preperation to get border vertices
        tricall=np.unique(tri[list0].flatten())
        sorted=np.argsort(np.linalg.norm(vert[tricall.astype(int)],axis=1))
        
        #and triangles
        dic={}
        verts=vert[tricall.astype(int)][sorted]
        for t in tricall:
            ind=int(np.where(np.all(vert[int(t)]==verts,axis=1))[0])
            dic[int(t)]=ind       
        

        #if there is only one surface
        if (np.all(maskc) or np.all(~maskc)): 
            #set triangles
            T=tri[list0]
            mesh.triangles = o3d.cpu.pybind.utility.Vector3iVector(T) 
            surf = vedo.mesh.Mesh([vert_def,T])
            surf.addPointArray(norm, "Normals")
            #border vertices and triangles
            border=o3d.geometry.PointCloud()
            border.points = o3d.utility.Vector3dVector(vert_def[tricall.astype(int)][sorted])
            dic={}
            verts=vert[tricall.astype(int)][sorted]
            for t in tricall:
                ind=int(np.where(np.all(vert[int(t)]==verts,axis=1))[0])
                dic[int(t)]=ind      
            with np.nditer(T, op_flags=['readwrite']) as it:
                    for ts in it:
                        ts[...]=dic[int(ts)]
            #sample points on surface
            pcd1 = mesh.sample_points_poisson_disk(sample_nr)#sample_points_uniformly(number_of_points=sample_nr) #latter seems to be about three orders of magnitude faster, but does not sample as nice
            #Store whole side independent of view
            if np.all(maskc):
                #calculate area 
                area1 = surf.area()
                o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])-1}.xyz', pcd1)
                o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])-1}_border.xyz', border)               
                np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])-1}_border.triangle",T)
                np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])-1}_border.normals",norm[tricall.astype(int)][sorted])
            else:
                #calculate area
                area2 = surf.area()
                o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])+1}.xyz', pcd1)
                o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])+1}_border.xyz', border)
                np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])+1}_border.triangle",T)
                np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])+1}_border.normals",norm[tricall.astype(int)][sorted])
            #if you want to see a visualization
            if show:
                #set triangles to the whole object
                mesh.triangles = o3d.cpu.pybind.utility.Vector3iVector(tri)
                #try to simulate camera positions as dots
                middle=np.mean(vert_def,axis=0)
                maxbound=np.max(np.abs(vert_def-middle))
                camera = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.array(CameraPositions)*maxbound*2+middle))
                #point sampled points according to prior or next
                if np.all(maskc): 
                    pcd1.paint_uniform_color([0,1,0])
                else:
                    pcd1.paint_uniform_color([1,0,0])
                #show mesh and points
                o3d.visualization.draw_geometries([mesh,pcd1,camera])

            #append sampled points to deformed vertices and create new vedo mesh for capturing
            samp = vedo.mesh.Mesh([np.concatenate((vert_def,np.asarray(pcd1.points))),tri])
            samp.addPointArray(np.concatenate((norm,np.zeros((sample_nr,3)))), "Normals")
            #store as object
            #samp = vedo.mesh.Mesh([vert_def,tri])
            #samp.polydata().GetPointData().SetActiveNormals("Normals")
            #vedo.io.write(samp,f'{folder_out}/{filename}.obj')
            #return samp

        #if there are two surfaces
        else:
            #determine mask for border vertices
            vertcall=vert[tricall.astype(int)][sorted]
            maskcall=vertcall[:,1]>meanc[1]
            #set triangles for prior surface
            T=tri[list0[maskc]]
            mesh.triangles = o3d.cpu.pybind.utility.Vector3iVector(T)
            surf = vedo.mesh.Mesh([vert_def,T])
            surf.addPointArray(norm, "Normals")
            #calculate area of prior side
            area1 = surf.area()
            #sample points on prior surface
            pcd1 = mesh.sample_points_poisson_disk(sample_nr)#.sample_points_uniformly(number_of_points=sample_nr)
            o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])-1}.xyz', pcd1)
            #border and triangles
            border1=o3d.geometry.PointCloud()
            border1.points = o3d.utility.Vector3dVector(vert_def[tricall.astype(int)][sorted][maskcall])
            o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])-1}_border.xyz', border1)
            dic={}
            verts=vert[tricall.astype(int)][sorted][maskcall]
            for t in T.flatten():
                ind=int(np.where(np.all(vert[int(t)]==verts,axis=1))[0])
                dic[int(t)]=ind   
            with np.nditer(T, op_flags=['readwrite']) as it:
                for ts in it:
                    ts[...]=dic[int(ts)]
            np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])-1}_border.triangle",T)
            np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])-1}_border.normals",norm[tricall.astype(int)][sorted][maskcall])
            #set triangles for next surface
            T=tri[list0[~maskc]]
            mesh.triangles = o3d.cpu.pybind.utility.Vector3iVector(T)
            surf = vedo.mesh.Mesh([vert_def,T])
            surf.addPointArray(norm, "Normals")
            #calculate area of next side
            area2 = surf.area()
            #sample points on next surface
            pcd2 = mesh.sample_points_poisson_disk(sample_nr)#.sample_points_uniformly(number_of_points=sample_nr)
            o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])+1}.xyz', pcd2)
            #border and triangles
            border2=o3d.geometry.PointCloud()
            border2.points = o3d.utility.Vector3dVector(vert_def[tricall.astype(int)][sorted][~maskcall])
            o3d.io.write_point_cloud(f'{folder_out}/{filename}_cutwith{int(filename[-1])+1}_border.xyz', border2)
            dic={}
            verts=vert[tricall.astype(int)][sorted][~maskcall]
            for t in T.flatten():
                ind=int(np.where(np.all(vert[int(t)]==verts,axis=1))[0])
                dic[int(t)]=ind   
            with np.nditer(T, op_flags=['readwrite']) as it:
                for ts in it:
                    ts[...]=dic[int(ts)]
            np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])+1}_border.triangle",T)
            np.savetxt(f"{folder_out}/{filename}_cutwith{int(filename[-1])+1}_border.normals",norm[tricall.astype(int)][sorted][~maskcall])
            #if you want to see a visualization
            if show:
                #paint prior side green
                pcd1.paint_uniform_color([0,1,0])
                #paint next side red
                pcd2.paint_uniform_color([1,0,0])
                #set triangles to the whole object
                mesh.triangles = o3d.cpu.pybind.utility.Vector3iVector(tri)
                #show mesh and points
                o3d.visualization.draw_geometries([mesh,pcd1,pcd2])
                
            #append sampled points to deformed vertices and create new vedo mesh for capturing
            samp = vedo.mesh.Mesh([np.concatenate((vert_def,np.asarray(pcd1.points),np.asarray(pcd2.points))),tri])
            samp.addPointArray(np.concatenate((norm,np.zeros((sample_nr*2,3)))), "Normals")
        
        #store the ids of the vertices
        samp.addPointArray(range(samp.NPoints()), "v_ids") 

        #work with camera to capture different perspectives
        
        #for each camera position
        for pos in CameraPositions:
            #create plotter
            vp = vedo.plotter.Plotter()
            #set camera position (there is more options too, but standard focal point seems to work well)
            vp.camera.SetPosition(pos)
            #add mesh to plotter
            vp.show(samp)#, resetcam=False)
            #get visible vertices from plotter https://vedo.embl.es/autodocs/content/vedo/pointcloud.html#visiblepoints
            m = vedo.pointcloud.visiblePoints(samp)
            vp.close()
            #get back the ids of the visible vertices
            ids = m.getPointArray("v_ids").astype(np.int) 
            #only consider sampled vertices
            sample_pointids=ids[ids>=len(vert)]
            #determine prior or next side by id, because we added prior first
            side=sample_pointids<(len(vert)+sample_nr)
            #if there only was the next side invert it
            if np.all(~maskc): side=~side
            #calculate minimum number of visible points to capture from viewpoint
            minnr=min_per/100*sample_nr
            #calculate number of visible points belonging to the cut to the prior side
            priornr=np.sum(side)
            #calculate number of visible points belonging to the cut to the next side
            nextnr=(len(side)-priornr)
            #if number of points on side is bigger than minimum and smaller than all points, store capture from this viewpoint 
            if (priornr>minnr) and (priornr<sample_nr): 
                #make point cloud
                vpts_prior = vedo.pointcloud.Points(samp.points()[sample_pointids[side]])
                #interpolate area
                area1v=int(np.around(area1*priornr/sample_nr))
                areas.append(area1v)
                #broad or narrow
                center=np.mean(vpts_prior.points(),axis=0)
                bn=np.mean(np.linalg.norm(vpts_prior.points()-center, axis=1))/np.sqrt(area1v)
                bna.append(bn)
                #store
                new_filename=f'{folder_out}/Views/{filename}_cutwith{int(filename[-1])-1}_view{pos}.xyz'
                vedo.io.write(vpts_prior, new_filename)
                csv_data.append([new_filename,f'{folder_out}/{filename[:-1]}{int(filename[-1])-1}_cutwith{filename[-1]}.xyz',filename[-1],pos,area1v,np.round(bn,4)])
            if (nextnr>minnr) and (nextnr<sample_nr):
                #make point cloud
                vpts_next = vedo.pointcloud.Points(samp.points()[sample_pointids[~side]])
                #interpolate area
                area2v=int(np.around(area2*nextnr/sample_nr))
                areas.append(area2v)
                #broad or narrow
                center=np.mean(vpts_next.points(),axis=0)
                bn=np.mean(np.linalg.norm(vpts_next.points()-center, axis=1))/np.sqrt(area2v)
                bna.append(bn)
                #store
                new_filename=f'{folder_out}/Views/{filename}_cutwith{int(filename[-1])+1}_view{pos}.xyz'
                vedo.io.write(vpts_next, new_filename)
                csv_data.append([new_filename,f'{folder_out}/{filename[:-1]}{int(filename[-1])+1}_cutwith{filename[-1]}.xyz',filename[-1],pos,area2v,np.round(bn,4)])
        toc = time.perf_counter()
        print(f"{filename} finished in {toc - tic:0.4f} seconds")

    #write view properties to csv file
    with open(f'{folder_out}/Views/Properties.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(csv_header)
        # write multiple rows
        writer.writerows(csv_data)

    print("Finished everything!")
    return areas, bna

In [ ]:
#Create Camera positions array, change steps as wanted
CameraPositions=[]
steps=np.arange(-1,1.5)
for x in steps:
    for y in steps:
        for z in steps:
            if np.any(np.abs([x,y,z]) == 1):
                CameraPositions.append([x,y,z])

In [ ]:
#Example, how to capture cuts for different shapes and gravities
folder="2 Deformation and Slicing/SimulationResults"
shapes=glob.glob(f"{folder}/*")
for shape in shapes:
    print(shape)
    gravities=glob.glob(f"{shape}/*")
    for grav in gravities: 
        print(grav)
        capture(show=0,CameraPositions=CameraPositions,folder_in=grav,folder_out=f"{grav}/Cut Sides 5000", sample_nr=5000)

## Recapturing process
Use if border data already exists

In [ ]:
import numpy as np
import vedo
import open3d as o3d
import glob
from pathlib import Path  
import time
import os
import csv

def recapture(folder_in,folder_out,CameraPositions,sample_nr=5000,min_per=5):
    """
    Function for capturing if border data is existent from an initial capturing run. Works even with dataset that has been made unique for downsampling, unlike the initial capturing process.

    Recaptures from different perspectives visible cutting surfaces by loading the border data from initial capturing, sampling the cutting surfaces and then capturing the visible sampled points as point clouds. 
    
    params
    ------
    folder_in : string
        Folder where the initial border captures are placed in
    folder_out : string
        Folder where to store the captured point clouds
    sample_nr : int [1,inf)
        Number of sampled points per cutting surface.
    min_per : float [0,100)
        Minimum needed visible surface in percent of whole surface for capturing point cloud from view.
    CameraPositions : Array/List (N,3)
        Array of [X, Y, Z] camera positions to capture from, focus is always on object. Only depends on the relation between the components, i.e. [100,0,0] is the same as [1,0,0] and [100,10,0] is the same as [10,1,0].  

    returns
    -------
    Stores point clouds from different views as .xyz files in Views subfolder. Also stores a Properties.csv with information on each view file. 
    """

    #create output directory, if it does not exist yet
    if not os.path.exists(folder_out):
        os.makedirs(folder_out)
    if not os.path.exists(f"{folder_out}/Views"):
        os.makedirs(f"{folder_out}/Views")
    areas=[] #to make histogram of captured areas
    bna=[] #to make histogram of broad narrow parameter
    #Lists to store additional data for views in csv file
    csv_header=["Filename","Target (Filename of other side of cut)", "Number of slice", "Camera Position", "Area", "Broad Narrow parameter"]
    csv_data=[]
    
    #go through all objects in input folder
    for obj in glob.glob(f"{folder_in}/*_border.xyz"):
        tic = time.perf_counter() #take time
        #load vertices initial, triangles, normals deformed and vertices deformed of cut surfaces
        filename=Path(obj).stem
        vert=np.loadtxt(obj)
        tri=np.loadtxt(f"{folder_in}/{filename}.triangle")
        norm=np.loadtxt(f"{folder_in}/{filename}.normals")

        #create mesh
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(vert)
        mesh.vertex_normals=o3d.utility.Vector3dVector(norm)
        mesh.triangles = o3d.cpu.pybind.utility.Vector3iVector(tri)
            
        #sample points on surface
        pcd1 = mesh.sample_points_poisson_disk(sample_nr)#sample_points_uniformly(number_of_points=sample_nr) #latter seems to be about three orders of magnitude faster, but does not sample as nice
        #Store whole side independent of view
        o3d.io.write_point_cloud(f'{folder_out}/{filename[:-7]}.xyz', pcd1)

        #calculate area 
        surf = vedo.mesh.Mesh([vert,tri])
        surf.addPointArray(norm, "Normals")
        area1 = surf.area()
               
        #append sampled points to deformed vertices and create new vedo mesh for capturing
        samp = vedo.mesh.Mesh([np.concatenate((vert,np.asarray(pcd1.points))),tri])
        samp.addPointArray(np.concatenate((norm,np.zeros((sample_nr,3)))), "Normals")
        
        #store the ids of the vertices
        samp.addPointArray(range(samp.NPoints()), "v_ids") 

        #load complete slice for occlusion
        vert_comp=np.loadtxt(f"{folder_in}/../Slices deformed/{filename[:-16]}.xyz")
        tri_comp=np.loadtxt(f"{folder_in}/../Slices/{filename[:-16]}.triangle")
        norm_comp=np.loadtxt(f"{folder_in}/../Slices deformed/{filename[:-16]}.normals")
        comp = vedo.mesh.Mesh([vert_comp,tri_comp])
        comp.addPointArray(norm_comp, "Normals")
        
        #work with camera to capture different perspectives
        
        #for each camera position
        for pos in CameraPositions:
            #create plotter
            vp = vedo.plotter.Plotter()
            #set camera position (there is more options too, but standard focal point seems to work well)
            vp.camera.SetPosition(pos)
            #add mesh to plotter
            vp.show(samp,comp)#, resetcam=False)
            #get visible vertices from plotter https://vedo.embl.es/autodocs/content/vedo/pointcloud.html#visiblepoints
            m = vedo.pointcloud.visiblePoints(samp)
            vp.close()
            #get back the ids of the visible vertices
            ids = m.getPointArray("v_ids").astype(np.int) 
            #only consider sampled vertices
            sample_pointids=ids[ids>=len(vert)]
            #calculate minimum number of visible points to capture from viewpoint
            minnr=min_per/100*sample_nr
            #calculate number of visible points belonging to the cut
            priornr=len(sample_pointids)
            #if number of points on side is bigger than minimum and smaller than all points, store capture from this viewpoint 
            if (priornr>minnr) and (priornr<sample_nr): 
                #make point cloud
                vpts_prior = vedo.pointcloud.Points(samp.points()[sample_pointids])
                #interpolate area
                area1v=int(np.around(area1*priornr/sample_nr))
                areas.append(area1v)
                #broad or narrow
                center=np.mean(vpts_prior.points(),axis=0)
                bn=np.mean(np.linalg.norm(vpts_prior.points()-center, axis=1))/np.sqrt(area1v)
                bna.append(bn)
                #store
                new_filename=f'{folder_out}/Views/{filename[:-7]}_view{pos}.xyz'
                vedo.io.write(vpts_prior, new_filename)
                if int(filename[-17])>int(filename[-8]):
                    matchnr=int(filename[-17])-1
                else:    
                    matchnr=int(filename[-17])+1
                csv_data.append([new_filename,f'{folder_out}/{filename[:-17]}{matchnr}_cutwith{filename[-17]}.xyz',filename[-17],pos,area1v,np.round(bn,4)])
            
        toc = time.perf_counter()
        print(f"{filename} finished in {toc - tic:0.4f} seconds")

    #write view properties to csv file
    with open(f'{folder_out}/Views/Properties.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(csv_header)
        # write multiple rows
        writer.writerows(csv_data)

    print("Finished everything!")
    return areas, bna

In [ ]:
#Create Camera positions array, change steps as wanted
CameraPositions=[]
steps=np.arange(-1,1.5)
for x in steps:
    for y in steps:
        for z in steps:
            if np.any(np.abs([x,y,z]) == 1):
                CameraPositions.append([x,y,z])

In [ ]:
#Example, how to recapture cuts for different gravities
defs=np.arange(0,4100,500)
folder="2 Deformation and Slicing/SimulationResults/DeformationCube (SameGravityAfterCut)"
for i,d in enumerate(defs):
    print(d)
    recapture(show=0,CameraPositions=CameraPositions,folder_in=f"{folder}/Gravity_{d}/CameraCaptures5000",folder_out=f"{folder}/Gravity_{d}/CameraCaptures2000N", sample_nr=2000)

## Histograms Broad Narrow and Area

In [ ]:
#Example how to load .csv file and analyze data
import pandas as pd
folder="2 Deformation and Slicing/SimulationResults/Cube/Gravity_1000/CameraCaptures5000"
df = pd.read_csv (f'{folder}/Views/Properties.csv')
bna=df["Broad Narrow parameter"]
narrow=np.sum(bna<1)
areas=df.Area
small=np.sum(areas<2000)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(areas);
plt.xlabel("Area [$distance^2$]")
plt.ylabel("Number of point clouds")
plt.title("Histogram of Areas")
plt.tight_layout()
plt.savefig(f"{folder}/AreasNew.pdf", dpi=400)

In [ ]:
plt.hist(bna);
plt.xlabel("Broad-Narrow parameter []")
plt.ylabel("Number of point clouds")
plt.title("Histogram of Broad-Narrow parameter")
plt.tight_layout()
plt.savefig(f"{folder}/BroadNarrowNew.pdf", dpi=400)

## Capture Complete objects and Slices
Function for capturing whole objects and slices instead of the cut surfaces. Should be run after cleaning up the UE4 data.

In [ ]:
import numpy as np
import vedo
import open3d as o3d
import glob
from pathlib import Path  
import time
import os
import csv

def capture_whole(type,undeformed,folder,min_per=40,CameraPositions=[[-150,0,0],[150,0,0],[0,-150,0],[0,150,0],[0,0,-150],[0,0,150]]):
    """
    Function for capturing whole objects and slices instead of the cut surfaces.
      
    params
    ------
    type : string ["Objects", "Slices"]
        Whether to capture objects or slices.
    folder : string
        Folder where Deformed, Slices and Slices Deformed folders are. Initial is one above
    min_per : float [0,100)
        Minimum needed visible surface in percent of whole surface for capturing point cloud from view.
    CameraPositions : Array/List (N,3)
        Array of [X, Y, Z] camera positions to capture from, focus is always on object. Only depends on the relation between the components, i.e. [100,0,0] is the same as [1,0,0] and [100,10,0] is the same as [10,1,0]. 

    returns
    -------
    Stores point clouds from different views as .xyz files in Views subfolder. Also stores a Properties.csv with information on each view file. 
    """

    #Lists to store additional data for views in csv file
    csv_header=["Filename", "Target", "Camera Position", "Area", "Broad Narrow parameter"]
    csv_data=[]
    csv_data_def=[]
    if type=="Slices":
        undeformed="Slices"
        deformed="Slices deformed"
    elif type=="Objects":
        undeformed="../Initial"
        deformed="Deformed"
    os.makedirs(f"{folder}/{undeformed}/Views", exist_ok=True)
    os.makedirs(f"{folder}/{deformed}/Views", exist_ok=True)

    #go through all objects in input folder
    for obj in glob.glob(f"{folder}/{deformed}/*.xyz"):
        tic = time.perf_counter() #take time
        #load vertices initial, triangles, normals deformed and vertices deformed
        filename=Path(obj).stem
        vert_def=np.loadtxt(obj)
        tri=np.loadtxt(f"{folder}/{undeformed}/{filename}.triangle")
        if type=="Slices": norm=np.loadtxt(f"{folder}/{deformed}/{filename}.normals")
        if undeformed: 
            vert=np.loadtxt(f"{folder}/{undeformed}/{filename}.xyz")
            surf = vedo.mesh.Mesh([vert,tri])
            #store the ids of the vertices
            surf.addPointArray(range(surf.NPoints()), "v_ids") 
            #calculate area 
            area = surf.area()

        #same for deformed
        surf_def = vedo.mesh.Mesh([vert_def,tri])
        if type=="Slices": surf.addPointArray(norm, "Normals")
        surf_def.addPointArray(range(surf_def.NPoints()), "v_ids")
        area_def = surf_def.area()

        
        #for each camera position
        for pos in CameraPositions:
            
            if undeformed:
                #create plotter for undeformed object
                vp = vedo.plotter.Plotter()
                #set camera position (there is more options too, but standard focal point seems to work well)
                vp.camera.SetPosition(pos)
                #add mesh to plotter
                vp.show(surf)#, resetcam=False)
                #get visible vertices from plotter https://vedo.embl.es/autodocs/content/vedo/pointcloud.html#visiblepoints
                m = vedo.pointcloud.visiblePoints(surf)
                vp.close()
                #calculate minimum number of visible points to capture from viewpoint
                minnr=min_per/100*len(vert)
                if len(m.points())>minnr:
                    areav=int(np.around(area*len(m.points())/len(vert)))
                    #broad or narrow
                    center=np.mean(m.points(),axis=0)
                    bn=np.mean(np.linalg.norm(m.points()-center, axis=1))/np.sqrt(areav)
                    #get back the ids of the visible vertices
                    ids = m.getPointArray("v_ids").astype(np.int) 
                    #store
                    new_filename=f'{folder}/{undeformed}/Views/{filename}_view{pos}.xyz'
                    np.savetxt(f"{new_filename[:-4]}_Correspondence.txt",ids)
                    vedo.io.write(m, new_filename)
                    csv_data.append([new_filename,filename,pos,areav,np.round(bn,4)])

            #create plotter for deformed object
            vp = vedo.plotter.Plotter()
            #set camera position (there is more options too, but standard focal point seems to work well)
            vp.camera.SetPosition(pos)
            #add mesh to plotter
            vp.show(surf_def)#, resetcam=False)
            #get visible vertices from plotter https://vedo.embl.es/autodocs/content/vedo/pointcloud.html#visiblepoints
            m = vedo.pointcloud.visiblePoints(surf_def)
            vp.close()
            minnr=min_per/100*len(vert)
            if len(m.points())>minnr:
                area_defv=int(np.around(area_def*len(m.points())/len(vert)))
                #broad or narrow
                center=np.mean(m.points(),axis=0)
                bn=np.mean(np.linalg.norm(m.points()-center, axis=1))/np.sqrt(area_defv)
                #get back the ids of the visible vertices
                ids = m.getPointArray("v_ids").astype(np.int) 
                #store
                new_filename=f'{folder}/{deformed}/Views/{filename}_view{pos}.xyz'
                np.savetxt(f"{new_filename[:-4]}_Correspondence.txt",ids)
                vedo.io.write(m, new_filename)
                csv_data_def.append([new_filename,filename,pos,area_defv,np.round(bn,4)])
            
        toc = time.perf_counter()
        print(f"{filename} finished in {toc - tic:0.4f} seconds")

    #write view properties to csv file
    if undeformed:
        with open(f'{folder}/{undeformed}/Views/Properties.csv', 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            # write the header
            writer.writerow(csv_header)
            # write multiple rows
            writer.writerows(csv_data)
    with open(f'{folder}/{deformed}/Views/Properties.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(csv_header)
        # write multiple rows
        writer.writerows(csv_data_def)

    print("Finished everything!")

In [ ]:
#Create Camera positions array, change steps as wanted
CameraPositions=[]
steps=np.arange(-1,1.5)
for x in steps:
    for y in steps:
        for z in steps:
            if np.any(np.abs([x,y,z]) == 1):
                CameraPositions.append([x,y,z])

In [ ]:
capture_whole("Slices","2 Deformation and Slicing/SimulationResults/Cube/Gravity_1000",CameraPositions=CameraPositions,min_per=40)

In [ ]:
#Example, how to capture objects for different shapes and gravities
defs=np.arange(0,4100,500)
folder="2 Deformation and Slicing/SimulationResults"
shapes=glob.glob(f"{folder}/*")
for shape in shapes:
    print(shape)
    gravities=glob.glob(f"{shape}/Gravity*")
    undeformed=1
    for grav in gravities: 
        print(grav)
        capture_whole("Objects",undeformed,grav,CameraPositions=CameraPositions,min_per=40)
        undeformed=0
        capture_whole("Slices",1,grav,CameraPositions=CameraPositions,min_per=40)

## Simple vedo plotting example

In [ ]:
samp=vedo.shapes.Sphere((100,100,100))
vp = vedo.plotter.Plotter()
#set camera position (there is more options too, but standard focal point seems to work well)
vp.camera.SetPosition([10,-2,0])
#add mesh to plotter
vp.show(samp)#, resetcam=False)
#get visible vertices from plotter
m = vedo.pointcloud.visiblePoints(samp)


In [ ]:
vp.show(samp,samp.wireframe())

In [ ]:
vedo.io.screenshot()

## Simple visualization of 3D data with vertices, triangles and normals if needed

In [ ]:
#Simple visualization with vertices, triangles and normals if needed. Surface might be in the other direction, hold left click and drag in some direction until you see the surface. Press W for wireframe

#file="d0_45_it5_sc300_dd0_5_main0"
file="2 Deformation and Slicing/SimulationResults/Cube/Gravity_1000/CameraCaptures5000/dr0_45_it5_sc100_sm0_5_main0_slice0_cutwith1_border"
tri=np.loadtxt(f"{file}.triangle")
vert_def=np.loadtxt(f"{file}.xyz")
norm=np.loadtxt(f"{file}.normals")

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vert_def)
mesh.triangles = o3d.cpu.pybind.utility.Vector3iVector(tri)
mesh.vertex_normals=o3d.utility.Vector3dVector(norm)

o3d.visualization.draw_geometries([mesh])